In [52]:
# Reload jupyter notebook when underlying files change
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## General instructions

1. Use the `requirements.txt` file to install the required packages

### Define the data we are going to use

In [71]:
import pathlib, json
DATA_DIR = pathlib.Path('/Users/edwardatkins/Downloads/TakeHomeTaskData')
data_path = DATA_DIR /  'the-internet-says-its-true_hollywood.json'
wave_file_loc = DATA_DIR / 'the-internet-says-its-true_hollywood.wav'
data = json.loads(data_path.read_text())
audio_data = open(wave_file_loc, 'rb').read()

## ACCESS 1: Python Package

In [72]:
from clipper import Clipper

In [73]:
# Initialize the object with the data
clipper = Clipper(audio_data, data, save_loc='/tmp/clip.wav')

In [74]:
# Run the process that identifies the best clip
result = clipper.run()

Calculating embeddings for entire text list
Calculating average embedding for entire document
Finding best sliding window
that many movies shot there . So I hope it's not that one , I'm going to be very embarrassed . I'm going to have to say Atlanta , I makes sense . Y'All you are correct , is at Landa . I only put defiance on the on the list because you're from there yeahs , you were born there , but there was a movie based in defiance because it was based on the book called the prize winner of defiance Ohio , it's great great film . When was that I wrote a pardon yeah , that's it starred Woody Harrelson and it's a great book . It's a memoir about a woman who she back in . I think the s and the s they didn't have a lot of money and she would enter these contests where you had to create little jingles and everything
5734 5884
Find better start time
Max gap:  1.5199999999999818
Max gap index:  5731


In [75]:
# Cut the audio file
clipper.cut_audio(result['window_start_token'], result['window_end_token'], clipper.item_list)

## ACCESS 2: Using FASTAPI
1. Start the FastAPI server using `uvicorn main:app --reload`

In [88]:
### Send post request to http://127.0.0.1:8000/clip with the transcript data

import requests
import json

with open(data_path) as f:
    data = json.load(f)

r = requests.post('http://127.0.0.1:8000/clip', json=data)
result = r.json(); result

{'_id': '3d689ff4-2153-4457-987c-031fd28b38a1'}

In [89]:
### Use the ID returned to upload the audio file if desired
url = f'http://127.0.0.1:8000/clip/upload_audio/{result["_id"]}'
r = requests.post(url, files={'file': open(wave_file_loc, 'rb')})
r.json()

{'_id': '3d689ff4-2153-4457-987c-031fd28b38a1'}

In [91]:
# Get the clip result
r = requests.get(f'http://127.0.0.1:8000/clip/get_text/{result["_id"]}')
r.json()

{'window_start_token': 5731,
 'window_end_token': 5884,
 'text': "they haven't had that many movies shot there . So I hope it's not that one , I'm going to be very embarrassed . I'm going to have to say Atlanta , I makes sense . Y'All you are correct , is at Landa . I only put defiance on the on the list because you're from there yeahs , you were born there , but there was a movie based in defiance because it was based on the book called the prize winner of defiance Ohio , it's great great film . When was that I wrote a pardon yeah , that's it starred Woody Harrelson and it's a great book . It's a memoir about a woman who she back in . I think the s and the s they didn't have a lot of money and she would enter these contests where you had to create little jingles and everything",
 'item_list': [{'start_time': 0.24,
   'end_time': 0.72,
   'alternatives': [{'confidence': '1.0', 'content': 'California'}],
   'type': 'pronunciation'},
  {'alternatives': [{'confidence': '1.0', 'content': '

In [92]:
# Get the audio clip, this currently assumes a local file system
# Ideally this would be uploaded to an accessible S3 Bucket or similar

r = requests.get(f'http://127.0.0.1:8000/clip/get_audio/{result["_id"]}')
r.json()

{'audio_loc': '/tmp/3d689ff4-2153-4457-987c-031fd28b38a1_clip.wav'}